In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning
from datetime import datetime

In [ ]:
df = pd.read_csv('../../../data/smooth_df.csv')

date_column = 'Date'
date_number_column = 'Date Number'
ili_rate_column = 'ILI Rate'
query_columns = [col for col in df.columns if col not in [date_column, date_number_column, ili_rate_column]]

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

print(df.shape)

In [ ]:
def custom_time_series_split(df, date_column):
    # Custom time series split based on years
    years = df[date_column].dt.year.unique()
    splits = [(years[5:i], years[i-5:i], years[i]) for i in range(10, len(years) - 4)]
    return splits

def get_train_test_split_data(X, y, train_start_date, test_start_date, test_end_date, corr_start_date, train_years, val_window):
    # Get the training and test data for a specific split
    train_indices = (X[date_column] >= train_start_date) & (X[date_column] < test_start_date)
    corr_indices = (X[date_column] >= corr_start_date) & (X[date_column] < test_start_date)
    test_indices = (X[date_column] >= test_start_date) & (X[date_column] <= test_end_date)

    X_train, y_train = X[train_indices], y[train_indices]
    X_corr, y_corr = X[corr_indices], y[corr_indices]
    X_test, y_test = X[test_indices], y[test_indices]

    for cv_year in train_years[val_window:]:
        print(f'{cv_year}-09-01', f'{cv_year+1}-08-31')

    # Construct cv folds where each fold correpsonds to a flu season in the training data 
    fold_indices = [
        (X_train[date_column] >= f'{cv_year}-09-01') & (X_train[date_column] <= f'{cv_year+1}-08-31')
        for cv_year in train_years[val_window:]
    ]

    return (X_train.iloc[:, 1:], y_train, X_test.iloc[:, 1:], y_test, X_corr.iloc[:, 1:], y_corr, fold_indices)

def standardize_data(X_train, X_test):
    # Standardize the data using the training data's mean and standard deviation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [ ]:
def get_correlation_df(X_corr, y_corr):
    correlation_scores = []
    for query_column in X_corr.columns:
        correlation = y_corr.corr(X_corr[query_column])
        correlation_scores.append((query_column, correlation))
    return pd.DataFrame(correlation_scores, columns=['Query', 'Correlation'])

def correlation_based_feature_selection(X_corr, y_corr, X_train, X_test, test_year, threshold):
    correlation_df = get_correlation_df(X_corr, y_corr).sort_values(by='Correlation', ascending=False).reset_index(drop=True)

    if threshold is not None:
        relevant_queries = correlation_df[correlation_df['Correlation'] >= threshold]['Query'].to_list()
    else:
        relevant_queries = correlation_df.iloc[:400]['Query'].to_list()

    X_train = X_train[relevant_queries]
    X_test = X_test[relevant_queries]

    print("number of features after correlation based fs: ", X_train.shape[1])
    print("X_train: ", X_train.shape, "X_test: ", X_test.shape)

    return X_train, X_test

In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def get_flu_season_folds(X_train, y_train, fold_indices):
    folds = []
    for val_index in fold_indices:
        X_train_fold, y_train_fold = X_train[~val_index], y_train[~val_index]
        X_val_fold, y_val_fold = X_train[val_index], y_train[val_index]
        folds.append((X_train_fold, y_train_fold, X_val_fold, y_val_fold))
    return folds

def train_evaluate_fold(alpha, l1_ratio, X_train_fold, y_train_fold, X_val, y_val):
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    model.fit(X_train_fold, y_train_fold)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100
    return mape, np.count_nonzero(model.coef_)

def manual_cross_validation(folds, check_min_weight):
    param_grid = {
        'alpha': np.concatenate(
            (np.arange(0.001, 0.01, 0.0005), np.arange(0.01, 0.099, 0.001), np.arange(0.1, 1.01, 0.01), np.array([2, 5, 7, 10]))
        ),
        'l1_ratio': [0.7]
    }
    results = []

    for alpha in param_grid['alpha']:
        for l1_ratio in param_grid['l1_ratio']:
            fold_results = []
            param_non_zero_weights = []
            for i, (X_train_fold, y_train_fold, X_val, y_val) in enumerate(folds):
                mae, _non_zero_weights = train_evaluate_fold(alpha, l1_ratio, X_train_fold, y_train_fold, X_val, y_val)
                fold_results.append(mae)
                param_non_zero_weights.append(_non_zero_weights)
            mean_non_zero_weights = np.mean(param_non_zero_weights)
            mean_mae = np.mean(fold_results)
            results.append({'alpha': alpha, 'l1_ratio': l1_ratio, 'mean_mae': mean_mae, "mean_non_zero_weights": mean_non_zero_weights})

    results = sorted(results, key=lambda x: x['mean_mae'])
    min_non_zero_weights = min(results, key=lambda x: x['mean_non_zero_weights'])
    max_non_zero_weights = max(results, key=lambda x: x['mean_non_zero_weights'])

    print("RESULTS: ", results)
    print("MIN NON-ZERO-WEIGHTS: ", min_non_zero_weights, "MAX NON-ZERO-WEIGHTS: ", max_non_zero_weights)

    result_ret = results if check_min_weight else min(results, key=lambda x: x['mean_mae'])

    return (result_ret, min_non_zero_weights['mean_non_zero_weights'], max_non_zero_weights['mean_non_zero_weights'])

def get_best_model(results, X_train, y_train):
    for index in range(len(results)):
        best_model = ElasticNet(alpha=results[index]['alpha'], l1_ratio=results[index]['l1_ratio'], max_iter=10000)
        best_model.fit(X_train, y_train)
        number_of_non_zero_weights = np.count_nonzero(best_model.coef_)
        if number_of_non_zero_weights >= 150:
            return best_model

def tune_elastic_net_model(X_train, y_train, fold_indices, check_min_weight):
    folds = get_flu_season_folds(X_train, y_train, fold_indices)
    
    if check_min_weight:
        results, min_non_zero_weights, max_non_zero_weights = manual_cross_validation(folds, check_min_weight)
        best_model = get_best_model(results, X_train, y_train)
    else:
        best_result, min_non_zero_weights, max_non_zero_weights = manual_cross_validation(folds, check_min_weight)
        best_model = ElasticNet(alpha=best_result['alpha'], l1_ratio=best_result['l1_ratio'], max_iter=10000)
        best_model.fit(X_train, y_train)

    print("Number of non zero weight queries: ", np.count_nonzero(best_model.coef_))
    return best_model, min_non_zero_weights, max_non_zero_weights

def evaluate_best_model(best_model, X_test, y_test):
    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print("MAE: ", mae)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    pearson_corr, _ = pearsonr(y_test, y_pred)
    return y_pred, mae, mape, pearson_corr

In [ ]:
def get_regularisers(alpha, l1_ratio):
    λ_1 = alpha * l1_ratio
    λ_2 = alpha * 0.5 * (1 - l1_ratio)
    return (λ_1, λ_2)

def add_average_row(df):
    # Calculate the average of 'MAE', 'Pearson_Correlation', and 'MAPE'
    mae_avg = round(np.mean(df['MAE']), 5)
    mae_std = round(np.std(df['MAE']), 5)
    pearson_corr_avg = round(np.mean(df['Pearson_Correlation']), 5)
    pearson_corr_std = round(np.std(df['Pearson_Correlation']), 5)
    mape_avg = round(np.mean(df['MAPE']), 5)
    mape_std = round(np.std(df['MAPE']), 5)
    alpha_average = round(np.mean(df['Alpha']), 5)
    l1_ratio_average = round(np.mean(df['L1_ratio']), 5)

    df.loc[len(df)] = {
        'Year': 'Average',
        'MAE': str(mae_avg) + ' +- ' + str(mae_std),
        'Pearson_Correlation': str(pearson_corr_avg) + ' +- ' + str(pearson_corr_std),
        'MAPE': str(mape_avg) + ' +- ' + str(mape_std),
        'Alpha': alpha_average,
        'L1_ratio': l1_ratio_average,
        'λ_1': '',
        'λ_2': '',
        'Min number of weights from cv': '',
        'Max number of weights from cv': '',
        'Number Of Non-Zero Weights': ''
    }

def train_elastic_net(queries, val_window=0, corr=False, threshold=None, check_min_weight=False):
    # Train an ElasticNet model on the provided DataFrame
    X = df[[date_column] + queries]
    y = df[ili_rate_column]

    splits = custom_time_series_split(df, date_column)

    model_performance = pd.DataFrame(
        columns=['Year', 'MAE', 'Pearson_Correlation', 'MAPE', 'Alpha', 'L1_ratio', 'λ_1', 'λ_2', 'Min number of weights from cv', 'Max number of weights from cv', 'Number Of Non-Zero Weights']
    )
    model_predictions = pd.DataFrame(columns=['Date', 'Actual_ILI_Rate', 'Predicted_ILI_Rate'])

    for train_years, corr_years, test_year in splits:
        print(f'TRAIN START YEAR: {train_years[0]}, CORR START YEAR: {corr_years[0]}, TEST YEAR: {test_year}-{test_year+1}')
        print('number of features: ', len(queries))

        # Get training and test data for the current split
        train_start_date = f'{train_years[0]}-09-01'
        corr_start_date = f'{corr_years[0]}-09-01'
        test_start_date = f'{test_year}-09-01'
        test_end_date = f'{test_year + 1}-08-31'

        X_train, y_train, X_test, y_test, X_corr, y_corr, fold_indices = get_train_test_split_data(X, y, train_start_date, test_start_date, test_end_date, corr_start_date, train_years, val_window)
        print("X_train: ", X_train.shape, "X_test: ", X_test.shape)

        if corr:
            X_train, X_test = correlation_based_feature_selection(X_corr, y_corr, X_train, X_test, test_year, threshold)

        selected_columns = X_train.columns
        X_train, X_test = standardize_data(X_train, X_test)

        # Tune and evaluate the ElasticNet model
        best_model, min_non_zero_weights, max_non_zero_weights = tune_elastic_net_model(X_train, y_train, fold_indices, check_min_weight)
        number_of_non_zero_weights = np.count_nonzero(best_model.coef_)
        
        y_pred, mae, mape, pearson_corr = evaluate_best_model(best_model, X_test, y_test)
        
        alpha, l1_ratio = best_model.get_params()['alpha'], best_model.get_params()['l1_ratio']
        λ_1, λ_2 = get_regularisers(alpha, l1_ratio)

        # Record the performance for this split
        model_performance.loc[len(model_performance)] = {
            'Year': f'{test_year}-{test_year + 1}',
            'MAE': round(mae, 5),
            'Pearson_Correlation': round(pearson_corr, 5),
            'MAPE': round(mape, 5),
            'Alpha': round(alpha, 5),
            'L1_ratio': round(l1_ratio, 5),
            'λ_1': round(λ_1, 5),
            'λ_2': round(λ_2, 5),
            'Min number of weights from cv': round(min_non_zero_weights),
            'Max number of weights from cv': round(max_non_zero_weights),
            'Number Of Non-Zero Weights': number_of_non_zero_weights
        }

        # Create a DataFrame of predicted ILI rates
        date_range = pd.date_range(start=test_start_date, periods=len(y_pred))
        iteration_predictions = pd.DataFrame({
            'Date': date_range,
            'Predicted_ILI_Rate': y_pred,
            'Actual_ILI_Rate': y_test,
        })

        model_predictions = pd.concat([model_predictions, iteration_predictions])
        print('\n\n')

    add_average_row(model_performance)
    return model_performance, model_predictions

In [ ]:
model_performance, model_predictions = train_elastic_net(query_columns, val_window=-3, corr=True, threshold=0.5, check_min_weight=True)
print(model_performance)
model_performance.to_csv(f'../../../model_results/nowcasting/elastic_net/correlation/cv_with_min_150_features/nowcasting_performance.csv')
model_predictions.to_csv(f'../../../model_results/nowcasting/elastic_net/correlation/cv_with_min_150_features/nowcasting_predictions.csv')

In [ ]:
filtered_queries = pd.read_csv('../../sentence_embedding_feature_selection/results/average.csv').iloc[:400]['Query'].to_list()
model_performance, model_predictions = train_elastic_net(filtered_queries, val_window=-3, check_min_weight=True)
print(model_performance)
model_performance.to_csv(f'../../../model_results/nowcasting/elastic_net/sentence_embedding/cv_with_min_150_features/nowcasting_performance.csv')
model_predictions.to_csv(f'../../../model_results/nowcasting/elastic_net/sentence_embedding/cv_with_min_150_features/nowcasting_predictions.csv')

In [ ]:
filtered_queries = pd.read_csv('../../sentence_embedding_feature_selection/results/average.csv').iloc[:1000]['Query'].to_list()
model_performance, model_predictions = train_elastic_net(filtered_queries, val_window=-3, corr=True, threshold=0.3, check_min_weight=True)
print(model_performance)
model_performance.to_csv(f'../../../model_results/nowcasting/elastic_net/hybrid/cv_with_min_150_features/nowcasting_performance.csv')
model_predictions.to_csv(f'../../../model_results/nowcasting/elastic_net/hybrid/cv_with_min_150_features/nowcasting_predictions.csv')

In [5]:
def define_periods(year):
    periods = {
        2014: [
            (datetime(2014, 9, 1), datetime(2014, 12, 1)),  # Onset
            (datetime(2014, 12, 2), datetime(2015, 1, 20)),  # Peak
            (datetime(2015, 1, 21), datetime(2015, 8, 31))  # Tail
        ],
        2015: [
            (datetime(2015, 9, 1), datetime(2015, 12, 30)),  # Onset
            (datetime(2015, 12, 31), datetime(2016, 3, 24)),  # Peak
            (datetime(2016, 3, 25), datetime(2016, 8, 31))  # Tail
        ],
        2016: [
            (datetime(2016, 9, 1), datetime(2016, 12, 1)),  # Onset
            (datetime(2016, 12, 2), datetime(2017, 1, 30)),  # Peak
            (datetime(2017, 1, 31), datetime(2017, 8, 31))  # Tail
        ],
        2017: [
            (datetime(2017, 9, 1), datetime(2017, 12, 30)),  # Onset
            (datetime(2017, 12, 31), datetime(2018, 2, 8)),  # Peak
            (datetime(2018, 2, 9), datetime(2018, 8, 31))  # Tail
        ],
        2018: [
            (datetime(2018, 9, 1), datetime(2018, 12, 28)),  # Onset
            (datetime(2018, 12, 29), datetime(2019, 2, 18)),  # Peak
            (datetime(2019, 2, 19), datetime(2019, 8, 31))  # Tail
        ]
    }

    return periods[year]

def analyse_flu_seasons(data, start_year, end_year):
    season_results = []
    
    for year in range(start_year, end_year):
        season_data = data[(data['Date'] >= datetime(year, 9, 1)) & (data['Date'] <= datetime(year + 1, 8, 31))]
        periods = define_periods(year)
        
        for start, end in periods:
            period_data = season_data[(season_data['Date'] >= start) & (season_data['Date'] <= end)]
            if not period_data.empty:
                y_test = period_data['Actual_ILI_Rate']
                y_pred = period_data['Predicted_ILI_Rate']
                mae = round(mean_absolute_error(y_test, y_pred), 5)
                period_name = f"{start.strftime('%Y-%m-%d')} - {end.strftime('%Y-%m-%d')}"
                season_results.append({'Flu Season': f'{year}-{year+1}', 'Period': period_name, 'MAE': mae})
    
    return pd.DataFrame(season_results)

for feature_selection in ['correlation', 'sentence_embedding', 'hybrid']:
    model_predictions = pd.read_csv(f'../../../model_results/nowcasting/elastic_net/{feature_selection}/cv_with_min_150_features/nowcasting_predictions.csv')
    model_predictions['Date'] = pd.to_datetime(model_predictions['Date'])
    
    period_performances = analyse_flu_seasons(model_predictions, 2014, 2019)
    period_performances.to_csv(f'../../../model_results/nowcasting/elastic_net/{feature_selection}/cv_with_min_150_features/nowcasting_period_performance.csv', index=False)